In [ ]:
# The code was removed by Watson Studio for sharing.

In [1]:
!conda install -c anaconda beautifulsoup4 -y
!conda install -c anaconda lxml -y
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    beautifulsoup4-4.8.2       |           py36_0         161 KB  anaconda
    certifi-2019.11.28         |           py36_1         157 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.2-py36_0      anaconda
    ca-certificates: 2020.1.1-0        --> 2020.1.1-0        anaconda
    certifi:         2019.11.28-py36_0 --> 2019.11.28-py36_1 anaconda
    openssl:         1.1.1e-

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import bs4 as bs
from bs4 import BeautifulSoup

In [3]:
import requests
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
#get the data using beautifulsoup from the link of wikipedia
soup=BeautifulSoup(url,'lxml')
match=soup.tbody
#print(match.prettify())

In [5]:
import sys
sys.getsizeof(match)

56

In [6]:
#make a dataframe from the data collected from Wikipedia
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
Postalcode = df["Postal code"].tolist()
Borough = df["Borough"].tolist()
Neighborhood = df["Neighborhood"].tolist()

In [7]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [8]:
#check size and type
df.dtypes
df.shape

(180, 3)

In [9]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. -- Dropped cells not assigned 
df = df[~df['Borough'].isin(['Not assigned'])]
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [10]:
df.shape #check new size of the database

(103, 3)

In [11]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, ...
#you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.loc[df['Neighborhood'] == 'NaN', 'Neighborhood'] = df['Borough'] 

In [13]:
# reset index
df = df.reset_index(drop=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
#check the final shape of the database
df.shape

(103, 3)

In [15]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    python_abi:    

In [16]:
#to not disrupt the previously created database, I am going to introduce a new db_2 which is the exact copy of the previous one but used for the part two of the assignment
df2=df.copy()
df2.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [17]:
df2['Postal Code']=df2['Postal code'] #this is added since the cvs file has a different name for the column
df2.drop(['Postal code'], axis=1, inplace=True)
df2= df2[['Postal Code', 'Borough','Neighborhood']]
df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
#**Geocoder gave me problem, so I used the csv file.**

for index, row in df.iterrows():
    postal_code=df2['Postal Code'].iloc[index]
    print('For', index, ' the postal code is', postal_code)
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

        
    latitude[index] = lat_lng_coords[0]
    longitude[index] = lat_lng_coords[1]    
    print('For', index, ' the lat and long are', latitude[index],' and ', longitude[index])

For 0  the postal code is M3A


KeyboardInterrupt: 

In [20]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
result = pd.merge(df2,df_data_1, on=['Postal Code'])
result.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [22]:
#import other useful libraries
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.21.0         | 58 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  en

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

In [345]:
#Coordinates of Toronto
from geopy.geocoders import Nominatim
address = 'Toronto, ON'
#Coordinates of Toronto for Google 43.6532° N, 79.3832° W
#latitude=43.6532
#longitude=-79.3832

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [346]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
#Pick the Borough with more neighborhoods 
result.groupby('Borough').count()

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6


In [36]:
toronto_data = result[result['Borough'].str.contains('Toronto')].reset_index(drop=True) #selecting only the Borough which have the word Toronto in their name
print(toronto_data.shape)
print(toronto_data.groupby('Borough').count())
toronto_data.head()


(39, 5)
                  Postal Code  Neighborhood  Latitude  Longitude
Borough                                                         
Central Toronto             9             9         9          9
Downtown Toronto           19            19        19         19
East Toronto                5             5         5          5
West Toronto                6             6         6          6


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [39]:
# The code was removed by Watson Studio for sharing.

In [40]:
#Explore Neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [42]:
#Make the function run for all Neighbours in XXX
LIMIT=100

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

print(toronto_venues.shape)
toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [50]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,77,77,77,77,77,77
Christie,18,18,18,18,18,18
Church and Wellesley,81,81,81,81,81,81
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [51]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


# Analyze Each Neighborhood that contains the word Toronto

In [59]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
#toronto_onehot.head()

(1693, 236)


In [60]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 236)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Business reply mail Processing CentrE,0.066667,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.012987,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012987,...,0.000000,0.000000,0.00,0.012987,0.000000,0.000000,0.012987,0.000000,0.000000,0.00
5,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.024691,0.012346,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012346,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012346,0.012346,0.00
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
8,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,Business reply mail Processing CentrE,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Light Rail Station,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Harbor / Marina,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Thai Restaurant,Burger Joint,Café,Ice Cream Shop,Spa,Middle Eastern Restaurant


# Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters since there are 5 Borough in the selection. Let's see if there is any homogenity depending on the borough or not.

In [62]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
toronto_grouped_clustering.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.066667,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.012987,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012987,0.0,...,0.0,0.0,0.0,0.012987,0.0,0.0,0.012987,0.0,0.0,0.0


In [63]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Theater,Mexican Restaurant,Breakfast Spot,Bakery,Restaurant,Café,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Gym,Diner,Music Venue,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop,Wings Joint,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Thai Restaurant,Bookstore,Pizza Place,Middle Eastern Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Clothing Store,Cocktail Bar,Beer Bar,Cosmetics Shop,Bakery,Diner
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store


In [203]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [191]:
#Examine Clusters
#Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#Cluster 1
Cluster1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Park,Theater,Mexican Restaurant,Breakfast Spot,Bakery,Restaurant,Café,Shoe Store
1,Downtown Toronto,0,Coffee Shop,Gym,Diner,Music Venue,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop,Wings Joint,Fried Chicken Joint
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Thai Restaurant,Bookstore,Pizza Place,Middle Eastern Restaurant
3,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Clothing Store,Cocktail Bar,Beer Bar,Cosmetics Shop,Bakery,Diner
4,East Toronto,0,Trail,Pub,Health Food Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Breakfast Spot
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Thai Restaurant,Burger Joint,Café,Ice Cream Shop,Spa,Middle Eastern Restaurant
7,Downtown Toronto,0,Grocery Store,Café,Park,Gas Station,Baby Store,Restaurant,Athletics & Sports,Italian Restaurant,Candy Store,Diner
8,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Bakery,Bar,Thai Restaurant,Clothing Store,Gastropub,Juice Bar,Salad Place
9,West Toronto,0,Bakery,Pharmacy,Bar,Pool,Bank,Supermarket,Grocery Store,Park,Gym / Fitness Center,Music Venue


In [192]:
#Cluster 2
Cluster2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,East Toronto,1,Sandwich Place,Gym,Food & Drink Shop,Liquor Store,Burrito Place,Italian Restaurant,Restaurant,Fast Food Restaurant,Steakhouse,Fish & Chips Shop
18,Central Toronto,1,Park,Lawyer,Swim School,Bus Line,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
20,Central Toronto,1,Gym,Hotel,Convenience Store,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Asian Restaurant,Park,Gas Station
21,Central Toronto,1,Trail,Sushi Restaurant,Park,Bus Line,Jewelry Store,Home Service,Doner Restaurant,Distribution Center,Dog Run,Women's Store
38,East Toronto,1,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Light Rail Station,Skate Park


In [193]:
#Cluster 3
Cluster3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Gym,Playground,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [194]:
#Cluster 4
Cluster4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,3,Park,Playground,Trail,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [195]:
#Cluster 5
Cluster5=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
Cluster5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Health & Beauty Service,Pool,Garden,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


# What can we conclude from this data?

From a first look, in the first cluster, many places where to eat are present: cafes, coffee shops, restaurants...
In the second clusters, there are still many place to eat out, but also parks, shops and gyms.
In the third cluster and in the fifth there are fewer resturants but also stores, such as electronics and department stores.
In the forth cluster, there are few restaurants but mostly parks and playgrounds.


In [340]:
from collections import Counter 
#cluster1
ClusterT=Cluster1.T
ClusterT.drop(['Cluster Labels','Borough'], inplace=True)
ClusterT.reset_index(inplace=True,drop=True)

#my_cols = set(Cluster1T.columns)
Counting = Counter(ClusterT[set(ClusterT.columns)].to_string().split())
#most_occur = Counting.most_common(10) 
most_common_words=pd.DataFrame(Counting.most_common()  , columns=['words','How often'])
indexNames = most_common_words[most_common_words['How often'] <= 1 ].index
symbols = most_common_words[most_common_words['words'] <= '&' ].index
# Delete these row indexes from dataFrame
most_common_words.drop(indexNames , inplace=True)
#most_common_words.drop(symbols , inplace=True)
most_common_words.head(8)

,words,How often
0,Restaurant,84
1,Shop,42
2,Coffee,25
3,Café,24
4,Bar,18
5,Store,17
6,Italian,13
7,Place,12


In [341]:
#cluster2
ClusterT=Cluster2.T
ClusterT.drop(['Cluster Labels','Borough'], inplace=True)
ClusterT.reset_index(inplace=True,drop=True)
Counting = Counter(ClusterT[set(ClusterT.columns)].to_string().split())
most_common_words=pd.DataFrame(Counting.most_common() , columns=['words','How often'])

indexNames = most_common_words[most_common_words['How often'] <= 1 ].index
symbols = most_common_words[most_common_words['words'] <= '&' ].index
# Delete these row indexes from dataFrame
most_common_words.drop(indexNames , inplace=True)
most_common_words.drop(symbols , inplace=True)
most_common_words.head(8)

,words,How often
0,Restaurant,11
1,Store,6
2,Place,5
3,Park,5
4,Shop,5
5,Food,3
7,Sandwich,2
8,Gym,2


In [343]:
#cluster3
ClusterT=Cluster3.T
ClusterT.drop(['Cluster Labels','Borough'], inplace=True)
ClusterT.reset_index(inplace=True,drop=True)
Counting = Counter(ClusterT[set(ClusterT.columns)].to_string().split())
most_common_words=pd.DataFrame(Counting.most_common()  , columns=['words','How often'])
indexNames = most_common_words[most_common_words['How often'] <= 1 ].index
symbols = most_common_words[most_common_words['words'] <= '&' ].index
# Delete these row indexes from dataFrame
most_common_words.drop(indexNames , inplace=True)
most_common_words.drop(symbols , inplace=True)
most_common_words.head()

,words,How often
0,Restaurant,5
1,Store,2


In [326]:
#cluster4
ClusterT=Cluster4.T
ClusterT.drop(['Cluster Labels','Borough'], inplace=True)
ClusterT.reset_index(inplace=True,drop=True)
Counting = Counter(ClusterT[set(ClusterT.columns)].to_string().split())
most_common_words=pd.DataFrame(Counting.most_common(10)  , columns=['words','How often'])
indexNames = most_common_words[most_common_words['How often'] <= 1 ].index
symbols = most_common_words[most_common_words['words'] <= '&' ].index
# Delete these row indexes from dataFrame
most_common_words.drop(indexNames , inplace=True)
most_common_words.drop(symbols , inplace=True)
most_common_words.head()

,words,How often
0,Restaurant,4


In [339]:
#cluster5
ClusterT=Cluster5.T
ClusterT.drop(['Cluster Labels','Borough'], inplace=True)
ClusterT.reset_index(inplace=True,drop=True)
Counting = Counter(ClusterT[set(ClusterT.columns)].to_string().split())
most_common_words=pd.DataFrame(Counting.most_common()  , columns=['words','How often'])

indexNames = most_common_words[most_common_words['How often'] <= 1 ].index
symbols = most_common_words[most_common_words['words'] <= '&' ].index
# Delete these row indexes from dataFrame
most_common_words.drop(indexNames , inplace=True)
#most_common_words.drop(symbols , inplace=True)
most_common_words.head()

,words,How often
0,Store,3
1,Restaurant,3
